In [3]:
import os
import itertools
from IPython.display import display

import numpy as np
import pandas as pd
import scipy.stats as sts

from tqdm import tqdm

from typing import Callable, Tuple, Any
from functools import partial

import matplotlib.pyplot as plt

pd.options.display.float_format = '{:.3f}'.format

In [4]:
STATISTIC = "./matrix/new_cat_jacc_sorted.csv"
BERT = "./matrix/scores_bert.csv"
MEDBERT = "./matrix/scores_medbert.csv"
DEEPSEEK = "./matrix/scores_DeepSeek-V3_mean.csv"
MLM = "./matrix/scores_mlm.csv"
YANDEX = "./matrix/scores_yandex_gpt5.csv"
DOC = "./matrix/scores_yandex_doc_search.csv"
QWEN = "./matrix/scores_qwen3-235b-a22b_0.csv"

In [5]:
statistic = pd.read_csv(STATISTIC).set_index("Unnamed: 0", drop=True)
bert = pd.read_csv(BERT).set_index("Unnamed: 0", drop=True)
medbert = pd.read_csv(MEDBERT).set_index("Unnamed: 0", drop=True)
deepseek = pd.read_csv(DEEPSEEK).set_index("Unnamed: 0", drop=True)
mlm = pd.read_csv(MLM).set_index("Unnamed: 0", drop=True)
yandex = pd.read_csv(YANDEX).set_index("Unnamed: 0", drop=True)
doc = pd.read_csv(DOC).set_index("Unnamed: 0", drop=True)
qwen = pd.read_csv(QWEN).set_index("Unnamed: 0", drop=True)

In [6]:
matrices = {
    "statistic": statistic,
    "bert": bert,
    "medbert": medbert,
    "deepseek": deepseek,
    "mlm": mlm,
    "yandex": yandex,
    "doc": doc,
    "qwen": qwen
}

LLMS = ["deepseek", "qwen", "yandex", "mlm"]
BASELINES = ["statistic", "medbert", "bert", "doc"]

In [7]:
for k, v in matrices.items():
    print(k)
    display(v.head())

statistic


,A01,A02,A03,A04,A05,A06,A07,A08,A09,A15,...,Z90,Z91,Z92,Z93,Z94,Z95,Z96,Z97,Z98,Z99
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
A01,1.000,0.040,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
A02,0.040,1.000,0.000,0.003,0.000,0.000,0.000,0.004,0.002,0.004,...,0.001,0.001,0.001,0.001,0.003,0.001,0.001,0.002,0.001,0.000
A03,0.000,0.000,1.000,0.001,0.000,0.000,0.013,0.000,0.002,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
A04,0.000,0.003,0.001,1.000,0.002,0.000,0.001,0.046,0.013,0.003,...,0.050,0.043,0.048,0.066,0.058,0.039,0.038,0.034,0.042,0.063
A05,0.000,0.000,0.000,0.002,1.000,0.000,0.000,0.007,0.004,0.000,...,0.001,0.001,0.001,0.001,0.002,0.001,0.001,0.001,0.001,0.001


bert


,A01,A02,A03,A04,A05,A06,A07,A08,A09,A15,...,Z90,Z91,Z92,Z93,Z94,Z95,Z96,Z97,Z98,Z99
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
A01,1.000,0.859,0.666,0.859,0.823,0.739,0.830,0.883,0.873,0.820,...,0.805,0.771,0.720,0.701,0.752,0.780,0.764,0.739,0.709,0.749
A02,0.859,1.000,0.778,0.973,0.888,0.865,0.942,0.934,0.914,0.822,...,0.834,0.823,0.804,0.821,0.804,0.788,0.850,0.856,0.834,0.814
A03,0.666,0.778,1.000,0.791,0.791,0.919,0.806,0.739,0.729,0.669,...,0.776,0.733,0.760,0.809,0.733,0.657,0.758,0.781,0.835,0.778
A04,0.859,0.973,0.791,1.000,0.898,0.869,0.958,0.949,0.910,0.811,...,0.844,0.828,0.842,0.841,0.817,0.790,0.861,0.881,0.864,0.833
A05,0.823,0.888,0.791,0.898,1.000,0.848,0.882,0.885,0.877,0.824,...,0.887,0.866,0.806,0.821,0.817,0.779,0.833,0.858,0.861,0.881


medbert


...

,A01,A02,A03,A04,A05,A06,A07,A08,A09,A15,...,Z90,Z91,Z92,Z93,Z94,Z95,Z96,Z97,Z98,Z99
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
A01,1.000,0.700,0.000,0.000,0.000,0.000,0.000,0.600,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
A02,0.700,1.000,0.700,0.600,0.000,0.000,0.000,0.800,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
A03,0.000,0.700,1.000,0.700,0.600,0.700,0.000,0.800,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
A04,0.000,0.600,0.700,1.000,0.900,0.000,0.700,0.800,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
A05,0.000,0.000,0.600,0.900,1.000,0.000,0.000,0.800,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


doc


,A01,A02,A03,A04,A05,A06,A07,A08,A09,A15,...,Z90,Z91,Z92,Z93,Z94,Z95,Z96,Z97,Z98,Z99
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
A01,1.000,0.743,0.781,0.768,0.758,0.784,0.779,0.759,0.760,0.771,...,0.624,0.644,0.632,0.550,0.612,0.563,0.587,0.607,0.651,0.564
A02,0.743,1.000,0.781,0.899,0.861,0.733,0.856,0.872,0.841,0.760,...,0.552,0.554,0.530,0.486,0.574,0.511,0.520,0.498,0.499,0.475
A03,0.781,0.781,1.000,0.749,0.735,0.848,0.810,0.718,0.716,0.714,...,0.619,0.591,0.650,0.622,0.620,0.525,0.612,0.613,0.611,0.552
A04,0.768,0.899,0.749,1.000,0.877,0.764,0.877,0.930,0.898,0.791,...,0.563,0.565,0.586,0.496,0.663,0.551,0.577,0.544,0.582,0.504
A05,0.758,0.861,0.735,0.877,1.000,0.728,0.816,0.805,0.831,0.789,...,0.640,0.604,0.562,0.486,0.600,0.499,0.508,0.514,0.535,0.574


qwen


,A00,A01,A02,A03,A04,A05,A06,A07,A08,A09,...,Z90,Z91,Z92,Z93,Z94,Z95,Z96,Z97,Z98,Z99
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
A00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
A01,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
A02,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
A03,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
A04,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [8]:
intersection = None
for i, k in enumerate(matrices):
    if i == 0:
        intersection = set(matrices[k].index)
    else:
        intersection = intersection.intersection(set(matrices[k].index))

In [9]:
intersection = sorted(list(intersection))

In [10]:
len(intersection)

1696

In [11]:
intersection[:10]

['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A15']

In [12]:
for k, v in matrices.items():
    matrices[k] = v.loc[intersection, intersection]

In [13]:
for k, v in matrices.items():
    print(k)
    display(v.head(3))

statistic


,A01,A02,A03,A04,A05,A06,A07,A08,A09,A15,...,Z90,Z91,Z92,Z93,Z94,Z95,Z96,Z97,Z98,Z99
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
A01,1.000,0.040,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
A02,0.040,1.000,0.000,0.003,0.000,0.000,0.000,0.004,0.002,0.004,...,0.001,0.001,0.001,0.001,0.003,0.001,0.001,0.002,0.001,0.000
A03,0.000,0.000,1.000,0.001,0.000,0.000,0.013,0.000,0.002,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


bert


,A01,A02,A03,A04,A05,A06,A07,A08,A09,A15,...,Z90,Z91,Z92,Z93,Z94,Z95,Z96,Z97,Z98,Z99
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
A01,1.000,0.859,0.666,0.859,0.823,0.739,0.830,0.883,0.873,0.820,...,0.805,0.771,0.720,0.701,0.752,0.780,0.764,0.739,0.709,0.749
A02,0.859,1.000,0.778,0.973,0.888,0.865,0.942,0.934,0.914,0.822,...,0.834,0.823,0.804,0.821,0.804,0.788,0.850,0.856,0.834,0.814
A03,0.666,0.778,1.000,0.791,0.791,0.919,0.806,0.739,0.729,0.669,...,0.776,0.733,0.760,0.809,0.733,0.657,0.758,0.781,0.835,0.778


medbert


...

,A01,A02,A03,A04,A05,A06,A07,A08,A09,A15,...,Z90,Z91,Z92,Z93,Z94,Z95,Z96,Z97,Z98,Z99
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
A01,1.000,0.700,0.000,0.000,0.000,0.000,0.000,0.600,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
A02,0.700,1.000,0.700,0.600,0.000,0.000,0.000,0.800,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
A03,0.000,0.700,1.000,0.700,0.600,0.700,0.000,0.800,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


doc


,A01,A02,A03,A04,A05,A06,A07,A08,A09,A15,...,Z90,Z91,Z92,Z93,Z94,Z95,Z96,Z97,Z98,Z99
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
A01,1.000,0.743,0.781,0.768,0.758,0.784,0.779,0.759,0.760,0.771,...,0.624,0.644,0.632,0.550,0.612,0.563,0.587,0.607,0.651,0.564
A02,0.743,1.000,0.781,0.899,0.861,0.733,0.856,0.872,0.841,0.760,...,0.552,0.554,0.530,0.486,0.574,0.511,0.520,0.498,0.499,0.475
A03,0.781,0.781,1.000,0.749,0.735,0.848,0.810,0.718,0.716,0.714,...,0.619,0.591,0.650,0.622,0.620,0.525,0.612,0.613,0.611,0.552


qwen


,A01,A02,A03,A04,A05,A06,A07,A08,A09,A15,...,Z90,Z91,Z92,Z93,Z94,Z95,Z96,Z97,Z98,Z99
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
A01,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.500,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
A02,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
A03,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [14]:
def compute_bootstrapped_score(
        y_test: np.array, 
        y_prob: np.array, 
        scorer: Callable[[np.array, np.array], Any], 
        m_sample: int = None, 
        stratum_vals: np.array =None
    ):
    assert not isinstance(y_test, pd.Series), "y_test should be np.array"
    assert not isinstance(y_prob, pd.Series), "y_prob should be np.array"

    idx = np.array(range(len(y_test)))
    if m_sample is None: m_sample = len(y_test) #bootstrap sample size
        
    if stratum_vals is not None: #select equal number of samples from each category
        idx_bs = [] 
        for val in set(stratum_vals):
            stratum_idx = idx[stratum_vals == val] 
            idx_bs += np.random.choice(stratum_idx, size=len(stratum_idx), replace=True).tolist()
    else:
        idx_bs = np.random.choice(idx, size=m_sample, replace=True)

    try:
        return scorer(y_test[idx_bs], y_prob[idx_bs]).statistic
    except Exception as e:
        print("WARNING: Bootstrapping failed for", scorer.func.__name__ if isinstance(scorer, partial) else scorer.__name__, "with error", e)
        return np.nan
    
def compute_ci(
    y_test: np.array, 
    y_prob: np.array, 
    stratum_vals: np.array = None, 
    n_bootstraps: int = 1000, 
    m_sample: int = None, 
    scorer: Callable[[np.array, np.array], Any] = sts.spearmanr,
    alpha: float = 0.05, #95% CI
    verbose: int = 1, 
    return_se: bool = False
):
    assert len(y_test) == len(y_prob), "y_test and y_prob should have the same lengths"
        
    scores = []
    if verbose > 0:
        print(f"Bootstrap scores computing for {scorer.func.__name__ if isinstance(scorer, partial) else scorer.__name__}...")
        for _ in tqdm(range(n_bootstraps)):
            scores.append(compute_bootstrapped_score(y_test, y_prob, scorer, stratum_vals=stratum_vals, m_sample=m_sample))
    else:
        for _ in range(n_bootstraps):
            scores.append(compute_bootstrapped_score(y_test, y_prob, scorer, stratum_vals=stratum_vals, m_sample=m_sample))
    scores = np.array(scores)
    
    nans_share = np.sum(np.isnan(scores).astype(int))/len(scores)
    if nans_share > 0.5: #empirical threshold, you can change it if you have better solution
        print(f"WARNING: There is {nans_share*100:.0f}% NaNs in bootstrapped scores for {scorer.func.__name__ if isinstance(scorer, partial) else scorer.__name__}")
        random_idxs = np.random.choice(list(range(len(y_test))), size=20, replace=False)
        print("       random 20 entries from y_test:", y_test[random_idxs])
        print("corresponging 20 entries from y_prob:", y_prob[random_idxs])
        if return_se:
            return np.nan, np.nan, np.nan
        else:
            return np.nan, np.nan
    
    estimation = np.nanmean(scores)
    se = np.nanstd(scores)
    perc = sts.norm.ppf(1 - alpha/2)
    e_perc = se * perc
    
    if verbose > 1:
        plt.figure(figsize=(4, 2.5))
        plt.hist(scores, bins=50)
        plt.axvline(x = estimation, color = 'tab:orange', label = 'mean')
        plt.axvline(x = estimation - e_perc, color = 'tab:red', label = f'mean - e_{1-alpha:.2f}')
        plt.axvline(x = estimation + e_perc, color = 'tab:red', label = f'mean + e_{1-alpha:.2f}')
        plt.show()
    
    if return_se:
        return estimation, e_perc, se
    else:
        return estimation, e_perc

In [21]:
spearmanr = pd.DataFrame(data=-np.ones((len(LLMS), len(BASELINES))), columns=BASELINES, index=LLMS)

In [23]:
# for method1, method2 in tqdm(itertools.combinations(matrices.keys(), 2)):
for llm, baseline in tqdm(itertools.product(LLMS, BASELINES)):
    print("Computing estimate for {} and {}".format(llm, baseline))
    spearmanr.loc[llm, baseline] = sts.spearmanr(matrices[llm].values.flatten(), matrices[baseline].values.flatten()).statistic
    spearmanr.to_csv("spearman.csv")

0it [00:00, ?it/s]

Computing estimate for deepseek and statistic


1it [00:00,  2.28it/s]

Computing estimate for deepseek and medbert


2it [00:00,  2.37it/s]

...

14it [00:05,  2.48it/s]

Computing estimate for mlm and bert


15it [00:06,  2.45it/s]

Computing estimate for mlm and doc


16it [00:06,  2.44it/s]


In [15]:
spearman_mean = pd.DataFrame(data=-np.ones((len(LLMS), len(BASELINES))), columns=BASELINES, index=LLMS)
spearman_se = pd.DataFrame(data=-np.ones((len(LLMS), len(BASELINES))), columns=BASELINES, index=LLMS)
spearman_e_perc = pd.DataFrame(data=-np.ones((len(LLMS), len(BASELINES))), columns=BASELINES, index=LLMS)

In [18]:
from multiprocessing import Pool

def _worker_pair(args):
    """Compute CI for a single (llm, baseline) pair. Top-level for pickling."""
    (llm, baseline, x_flat, y_flat, kwargs) = args
    # call your existing function (single-process bootstrap inside)
    r, e_perc, se = compute_ci(
        x_flat, y_flat,
        return_se=True,
        **kwargs
    )
    return llm, baseline, r, e_perc, se

if __name__ == "__main__":  # important for Windows / notebooks
    # prepare outputs
    spearman_mean   = pd.DataFrame(-np.ones((len(LLMS), len(BASELINES))), columns=BASELINES, index=LLMS)
    spearman_se     = pd.DataFrame(-np.ones((len(LLMS), len(BASELINES))), columns=BASELINES, index=LLMS)
    spearman_e_perc = pd.DataFrame(-np.ones((len(LLMS), len(BASELINES))), columns=BASELINES, index=LLMS)

    # build tasks (pre-flatten to avoid doing it in workers repeatedly)
    common_kwargs = dict(
        scorer=sts.spearmanr,
        n_bootstraps=1000,   # adjust as you like
        m_sample=None,
        stratum_vals=None,
        alpha=0.05,
        verbose=0            # silence per-worker prints
    )
    tasks = []
    for llm, baseline in itertools.product(LLMS, BASELINES):
        x_flat = matrices[llm].values.ravel()
        y_flat = matrices[baseline].values.ravel()
        tasks.append((llm, baseline, x_flat, y_flat, common_kwargs))

    # run in parallel across pairs
    n_jobs = None  # or an int like 8
    with Pool(processes=n_jobs) as pool:
        for llm, baseline, r, e_perc, se in tqdm(pool.imap_unordered(_worker_pair, tasks, chunksize=1)):
            spearman_mean.loc[llm, baseline]   = r
            spearman_se.loc[llm, baseline]     = se
            spearman_e_perc.loc[llm, baseline] = e_perc
    
    spearman_mean.to_csv("spearman_mean.csv")
    spearman_se.to_csv("spearman_se.csv")
    spearman_e_perc.to_csv("spearman_e_perc.csv")

16it [13:17, 49.85s/it]


In [24]:
spearmanr

,statistic,medbert,bert,doc
deepseek,0.006,0.024,-0.022,0.139
qwen,0.071,0.029,0.029,0.093
yandex,0.070,0.070,0.057,0.093
mlm,-0.003,0.003,0.002,0.007


In [19]:
spearman_mean

,statistic,medbert,bert,doc
deepseek,0.006,0.024,-0.022,0.139
qwen,0.071,0.029,0.029,0.093
yandex,0.070,0.070,0.056,0.093
mlm,-0.003,0.003,0.002,0.007


In [20]:
spearman_se

,statistic,medbert,bert,doc
deepseek,0.001,0.001,0.001,0.001
qwen,0.001,0.001,0.001,0.001
yandex,0.001,0.001,0.001,0.001
mlm,0.001,0.001,0.001,0.001
